In [ ]:
from csv import DictReader
import os
import pandas as pd

from osemosys_global.OPG_powerplant_data import get_data, create_generators

In [ ]:
COUNTRIES = os.path.join('data', 'countries.csv')
INPUT_DATA = os.path.join('..', '..', 'osemosys_global', 'data')
OUTPUT_PATH = os.path.join('model_data')

In [ ]:
df, df_dict = get_data(INPUT_DATA)

In [ ]:
df.head()

In [ ]:
df_op_life = pd.read_csv(os.path.join(INPUT_DATA, "operational_life.csv"))
df_tech_code = pd.read_csv(os.path.join('..', 'data', "techmap.csv"))
df_gen_2 = create_generators(df, df_dict, 2015, df_op_life, df_tech_code)
df_gen_2["country_code"] = df_gen_2["country_code"].str[0:3]

In [ ]:
res_cap_cols = [
        "country_code",
        "powerplant",
        "tech_code",
        "total_capacity",
        "start_year",
        "retirement_year_model",
        "operational_life",
    ]

df_res_cap = df_gen_2[res_cap_cols]

In [ ]:
map_tech_codes = pd.read_csv(os.path.join('..', 'data', 'tech_name_map.csv'), 
                             dtype='str',
                             na_filter=False
                             ).set_index('plexos').to_dict()


In [ ]:
# Convert total capacity from MW to GW
df_res_cap['total_capacity'] = df_res_cap['total_capacity'].div(1000)

sand_tech = df_res_cap['tech_code'].apply(lambda x: map_tech_codes['sand'][x])

suffix = df_res_cap['tech_code'].apply(lambda x: map_tech_codes['suffix'][x])

df_res_cap["group"] = 'PWR' + sand_tech + suffix

# Allocate tech codes for hydro
# PWRHYD001	Large Hydropower Plant (Dam) (>100MW)
# PWRHYD002	Medium Hydropower Plant (10-100MW)
# PWRHYD003	Small Hydropower Plant (<10MW)
# PWRHYD004	Off-grid Hydropower

large_hydro = (df_res_cap['tech_code'] == 'HYD') & (df_res_cap['total_capacity'] >= 0.100)

medium_hydro = (df_res_cap['tech_code'] == 'HYD') & \
               (df_res_cap['total_capacity'] >= 0.010) & \
               (df_res_cap['total_capacity'] < 0.1)
small_hydro = (df_res_cap['tech_code'] == 'HYD') & \
              (df_res_cap['total_capacity'] >= 0) & \
              (df_res_cap['total_capacity'] < 0.01)

df_res_cap.loc[large_hydro, "group"] = 'PWR' + sand_tech + "001"
df_res_cap.loc[medium_hydro, "group"] = 'PWR' + sand_tech + "002"
df_res_cap.loc[small_hydro, "group"] = 'PWR' + sand_tech + "003"

df_res_cap['powerplant'] = df_res_cap['powerplant'].str.split('_', expand=True).loc[:, 2]

In [ ]:
def filter_by_country(df, country_code):
    filterbycountry = df['country_code'] == country_code
    return df[filterbycountry]

In [ ]:
filter_columns = ['lookup', 'country_code', 'group', 'id', 'powerplant', 'operational_life',  'start_year', 'total_capacity']
column_map = {
    'operational_life': 'pp_life_time',  
    'start_year': 'year_built', 
    'total_capacity': 'capacity'
}
df_res_cap['id'] = df_res_cap.groupby(by=['country_code', 'group']).cumcount()
df_res_cap['lookup'] = df_res_cap['country_code'] + df_res_cap['id'].apply(str) + df_res_cap['group']
df_res_cap[filter_columns].rename(columns=column_map).to_csv('residual_capacity.csv', index=False)

In [ ]:
countries = df_res_cap.country_code.unique()

In [ ]:
for country in countries:
    df = filter_by_country(df_res_cap, country)
    df[filter_columns].rename(columns=column_map).to_csv("resid_cap_{}.csv".format(country), index=False)